In [1]:
import os


In [2]:
# to verify present folder
%pwd

'/home/cdot/PycharmProjects/ChickeDecease/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/cdot/PycharmProjects/ChickeDecease'

##### Data ingestion related config
##### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
# Entity Name
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfiguratoinManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
                                                    source_URL = config.source_URL,
                                                    local_data_file=config.local_data_file,
                                                    unzip_dir=config.unzip_dir)
        
        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_URL,
                                                    filename = self.config.local_data_file)
            
            logger.info(f"{filename} is download! with following info: \n{headers}")
        else:
            logger.info(f"File Already exists of size: {get_size(Path(self.config.local_data_file))}")
            

    def extract_zip_file(self):
        """
        zip file path is: str
        Extract the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [16]:
try:
    config = ConfiguratoinManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-11-23 14:13:29,740: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-23 14:13:29,748: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-23 14:13:29,750: INFO: common: Created directory at: artifacts]
[2023-11-23 14:13:29,751: INFO: common: Created directory at: artifacts/data_ingestion]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/data.zip'